# Data Acquisition

Time to talk about how we go about getting real data.  There's tons of different ways, and new ones being created all of the time.  These are by far the most popular for our domain.  Some require specialized infrastructures and environements, and others are great for learning and testing.  For our purposes, we're going to focus on the learning and testing aspects using real-world datasets publically available.

We'll cover the following for this section.

- Downloading files
- Database/Warehouse
- APIs

<h3>Downloading Files</h3>

By far the most common way you'll get data is by simple downloading of your files, until you start dealing with more sophisticated production ready infrastructures in the real-world anyway.  There are tons of different file extensions you will run into in the wild so there's no hope we could be exhaustive here, but we can review a couple of the most common ones which should cover you for the majority of use cases.

In practice, you'll occasionally run into new extensions you've never worked with before, and even errors running the most basic ingestion methods like `pd.read_csv()` due to encoding issues or unknown formatting problems in the file.  Python package documentation, <a href="https://stackoverflow.com/">stackoverflow</a>, and Google will continue to be your best friends when you run into errors or new files types.  You can generally bet that someone else has run into the same problem you're having and that the Q&A is out on the internet somewhere.

The workhorse of this acquisition group is overwhelmingly going to be our trusty <a href="https://pandas.pydata.org/docs/index.html">Pandas</a> library.  It can be leveraged for a huge variety of different types of files and is extremely flexible.

Off we go!

In [ ]:
import pandas as pd

<h5>CSV/Excel/TXT Files</h5>

Businesses run on Excel.  I can't say this enough.  It is the most common data storage and analytics tool for a reason.  It simply works and is super intuitive.  Excel puts some very sophisticated tools in the hands of non-technical resources, and it's the lowest and cheapest bar to entry for analytics.  Most companies will use it for these reasons, so skills are transferable from company to company and there's no lock out due to commercial licensing.

We already saw this in the {doc}`../Chapter5/data_types` section, but let's again read in a CSV/Excel file from our Github storage.  To do so we'll use pandas `pd.read_csv()` method.  This is an _extremely_ common way to read in data as you'll quickly learn.  Here we're pointing to our Github repository, but you will most often have your files in local storage on your computer so you'll just point the URL path to your working directory or local file.  Simple as that.

In [ ]:
# Read in Davis height/weight data from Github as a .csv file
url = 'https://github.com/bradybr/practical-data-science-and-ml/blob/main/datasets/Davis.csv?raw=true'
dat = pd.read_csv(url)
dat.head(10)

There are a ton of parameters with this function, so definitely read the documentation and do some Googling when you run into any difficulties.  For example, let's see what happens if we read in our file as a "tab-delimited" text file.  To do that we'll use the separator argument `sep = "\t"` in `pd.read_csv()` call.

In [ ]:
# Read in Davis height/weight data from Github as a .txt file
url = 'https://github.com/bradybr/practical-data-science-and-ml/blob/main/datasets/Davis.csv?raw=true'
dat = pd.read_csv(url, sep = "\t")
dat.head(10)

Interesting.  It looks like we do need to understand if our file is comma, tab, semicolon, or otherwise separated.  If our file was semicolon delimited, we would have written `sep = ";"`, but as it turns out, this one matched the default with comma separated so it was able to parse it out into a dataframe as-is.

If you have a true .xls file instead of a .csv file, then check out `pd.read_excel()` function, which handles the specialities which come with the more complex file type.

<h5>XML/HTML Tables</h5>

Reading data from the internet can often be very finicky and challenging.  Technology often gets involved, and you also need to know a little bit about the language of the internet: HTML.  

The most painful part is that a method which worked yesterday can easily break tomorrow.  Sites can change their HTML code anytime so you always have to stay on your toes.  It's also a burden on the sites resources you're trying to reach so it's quite common to have your IP address blocked if you abuse requesting information too often.  Specific types of sites are more sensitive to this than others, specifically ones that believe their data is a competitive advantage or proprietary in some way.  There are constant legal battles going on regarding whether or not the data belongs in the public domain after it's been published.

To illustrate, we'll read in a standing table from Wikipedia that shouldn't give us any trouble.  The code below queries a specific Wikipedia page listing all of the public companies currently listed in the <a href="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies">S&P500</a>.

In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import certifi
import urllib
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

# URL path
url = ("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

# Open URL & parse table content
html = urllib.request.urlopen(url, cafile = certifi.where())
soup = BeautifulSoup(html, 'html.parser')
tables = soup.findAll("table")

# Convert to dataframe
dat = pd.read_html(StringIO(str(tables[0])))[0]
dat

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


This example is taking advantage of a known "table" structure on a webpage.  Often however you'll need parse an entire page and crawl through the HTML nodes by parent, children, and various tags to find exactly what you're looking for.  This is referred to as _web scraping_ and _parsing_.  You will definitely need to do some googling and researching a bit with lots of trail and error when you begin to work with scraping web data for real.

There's much more you can do with the topic of web scrapping and accessing XML and HTML structures, but this is a bit beyond the scope of our lessons.  If you're interested, definitely check out Beautiful Soup, Selenium, and Scrappy libraries, just to name a few...

<h5>JSON Data</h5>

Closely coupled with internet data is the JSON structure type.  JSON stands for JavaScript Object Notation, and is a simple and convenient way to store and transport data in a text format.  You may run into this one with websites who choose to make their data publically available for download.  It's also very common to send and receive data this way in industrialized cloud infrastructures within large companies.

Below we'll retrieve the <a href="https://www.eia.gov/outlooks/steo/data/browser/">US Department of Energy's Short-Term Engergy Outlook</a> for Petroleum using their API call which returns data in a JSON format.  You'll need to sign up for your own API Key password if you'd like to try and replicate yourself.

Notice how the data returned in a format that looks very much like a Python dictionary?  Pretty cool, huh?  The JSON format is very close to native Python language and should hopefully be pretty easy to deal with after you've access it.

In [21]:
pw = '73aa6102f0b10664045a49a5f9789e24'

In [ ]:
import json
import requests

url = "https://api.eia.gov/v2/petroleum/move/wkly/data/?frequency=weekly&data[0]=value&facets[series][]=WCREXUS2&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=" + pw + "&start=2024-01-01"
json.loads(requests.get(url).text)

<h5>Database/Warehouse Data</h5>

Retrieving data stored in some kind of structured relational database is also extremely common.  You may have a free version like MySQL setup for your personal use, or maybe a more powerful commerical cloud based offering like Microsoft Azure and Data Lake.  This is a deep and specific subject due to the techologies, installations, and authorizations required, which are also a bit outside of the scope for these lessons.

Since we'll be dealing mostly with downloading files or accessing data via Python libraries, we'll leave this as an exercise for the reader to investigate further in his or her own time.  Below is the link to the popular and free MySQL offering where you can read more about connecting with Python.

And remember when we suggested learning the SQL language might be a good thing when discussing programming languages?  This is where it will come into play.  These relational databases are extremely common for warehousing enterprise data, and SQL is by far the most popular programming language to work with these data repositories.

<a href="https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html">MySQL Python Connector</a>

```{figure} ../images/mysql_connector.png
---
width: 900px
name: mysql-connector-fig
---
MySQL Python Connector Introduction documentation
```

<h5>APIs</h5>

We've actually just seen an example of an API call to retrieve data from the Department of Energy.  API stands for Application Programming Interface, and are standard protocols which allow programs and applications to communicate with each other.  They are essentially end-to-end connection points that have been set up to faciliate the transfer of data using request calls and responses.

The DOE call above is just one example, however API's have become very common with the explosion of business monetization of data as a core competency.  Social media sites like Twitter, Facebook, and the like, as well as other companies working with popular consumable data such as weather, stock prices, etc. are commonly utilizing this technology today.  Companies typically open these up to what are known as the "Developer" community.  You can sign up for an account and receive API Keys and authentication tokens which allow you to begin accessing data straight away.  Because this data is often a revenue stream for the provider, unfortunatly you will frequently run into a paywall for good data though.  Hard to blame them, seeing as they've incurred all of the expenses towards collecting and hosting the data, so it does seem only fair that they're compensated for it somehow.

We won't go into any more detail here since we've already shown an example and due to needing to create developer profiles.  Our work for this course will be mostly with downloaded files or through Python packages.